In [ ]:
import os
import pandas as pd
import numpy as np

FOLDER_PATH = os.getcwd()
FILE_NAME = "\\Traffic_Volume_Counts__2014-2019_.csv"
FILE_PATH = FOLDER_PATH + FILE_NAME

traffic_orig = pd.read_csv(FILE_PATH)

traffic = traffic_orig.copy()

traffic["Segment ID"] = traffic["Segment ID"].astype(int)
traffic["Is_Weekend"] =  pd.to_datetime(traffic["Date"]).dt.day_name().isin(['Saturday', 'Sunday'])
traffic["Season"] =  pd.to_datetime(traffic["Date"]).dt.month % 12 // 3 + 1

traffic_hour_column_names = ['12:00-1:00 AM', '1:00-2:00AM', '2:00-3:00AM', '3:00-4:00AM',
       '4:00-5:00AM', '5:00-6:00AM', '6:00-7:00AM', '7:00-8:00AM',
       '8:00-9:00AM', '9:00-10:00AM', '10:00-11:00AM', '11:00-12:00PM',
       '12:00-1:00PM', '1:00-2:00PM', '2:00-3:00PM', '3:00-4:00PM',
       '4:00-5:00PM', '5:00-6:00PM', '6:00-7:00PM', '7:00-8:00PM',
       '8:00-9:00PM', '9:00-10:00PM', '10:00-11:00PM', '11:00-12:00AM']

# renaming traffic hours
traffic = traffic.rename(columns=dict(zip(traffic_hour_column_names, list(range(24)))))

cols = ['ID', 'Segment ID', 'Roadway Name', 'Date', 'Is_Weekend', 'Season']

traffic_v1 = traffic.melt(
    id_vars = cols,
    value_vars = list(range(24)),
    var_name = 'Hour',
    value_name = 'Traffic_Volume'
)

# the main purpose of this is because it seems like the data-table has split the road across 2 rows in terms of direction. 
# But we don't really care if the traffic in 1 side is different from the traffic on the other side heading the opposite 
# direction. To us, it's the same road, so we combine the traffic on either sides together.

traffic_v2 = traffic_v1.groupby(cols + ["Hour"], as_index=False)["Traffic_Volume"].sum()

# Finally, the hours were binned together into groups of 3; therefore, a single 24-hour day is now just 8 chunks.
# these 3-hour subgroups were aggregated by mean traffic volume, rounded to 2 decimal places
cols = ["Segment ID", "Is_Weekend", "Season"]

traffic_v2["3_Hr_Intvl"] = ((traffic_v2["Hour"] // 3) + 1)*3
traffic_v3 = traffic_v2.groupby(cols + ["3_Hr_Intvl"], as_index=False)["Traffic_Volume"].mean().round(2)

FILE_NAME = "\\nyc_traffic_3hrInterval.csv"
FILE_PATH = FOLDER_PATH + FILE_NAME
traffic_v3.to_csv(FILE_PATH)